In [3]:
import os
print("Current working directory:", os.getcwd())


Current working directory: /content


In [12]:
# To create the 'critical' directory with subdirectories and sample files
import os

base_dir = "/content/critical"
sub_dirs = ["lab1", "lab2", "lab3"]

if not os.path.exists(base_dir):
    os.makedirs(base_dir)

for sub_dir in sub_dirs:
    sub_dir_path = os.path.join(base_dir, sub_dir)
    os.makedirs(sub_dir_path, exist_ok=True)
    with open(os.path.join(sub_dir_path, "sample_file.txt"), "w") as f:
        f.write("This is a sample file for encryption.")

print("Directory structure created successfully in /content.")


Directory structure created successfully in /content.


In [5]:
pip install pycryptodome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.0 MB/s eta 0:00:00


In [13]:

# AES encryption has been implemented to encrypt all the files within the critical directory.
import os
import hashlib
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

def encrypt_file(file_path, key):
    """Encrypts a file using AES in CBC mode."""
    try:
        cipher = AES.new(key, AES.MODE_CBC)
        with open(file_path, "rb") as f:
            data = f.read()

        pad_length = 16 - (len(data) % 16)
        data += bytes([pad_length]) * pad_length

        encrypted_data = cipher.encrypt(data)

        with open(file_path + ".enc", "wb") as f:
            f.write(cipher.iv)
            f.write(encrypted_data)

        os.remove(file_path)
        print(f"Encrypted file: {file_path}")
    except Exception as e:
        print(f"Error encrypting {file_path}: {e}")


def decrypt_file(file_path, key):
    """Decrypts a file using AES in CBC mode."""
    try:
        with open(file_path, "rb") as f:
            iv = f.read(16)
            encrypted_data = f.read()

        cipher = AES.new(key, AES.MODE_CBC, iv=iv)
        decrypted_data = cipher.decrypt(encrypted_data)

        padding_length = decrypted_data[-1]
        decrypted_data = decrypted_data[:-padding_length]

        with open(file_path[:-4], "wb") as f:
            f.write(decrypted_data)

        os.remove(file_path)
        print(f"Decrypted file: {file_path}")
    except Exception as e:
        print(f"Error decrypting {file_path}: {e}")


def hash_file(file_path):
    """Calculates the SHA-256 hash of a file."""
    hasher = hashlib.sha256()
    with open(file_path, "rb") as f:
        while True:
            chunk = f.read(4096)
            if not chunk:
                break
            hasher.update(chunk)
    return hasher.hexdigest()

# The Example usage:
aes_key = get_random_bytes(32)
base_dir = "/content/critical"

for root, dirs, files in os.walk(base_dir):
    for filename in files:
        file_path = os.path.join(root, filename)
        if os.path.isfile(file_path):
            encrypt_file(file_path, aes_key)
            encrypted_file_path = file_path + ".enc"
            if os.path.exists(encrypted_file_path):
                encrypted_file_hash = hash_file(encrypted_file_path)
                print(f"Encrypted file: {encrypted_file_path}, SHA-256: {encrypted_file_hash}")



Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc, SHA-256: 22b382edbf855d663219ce1fcb03646aed84b42f9926f297f7a349be6fecd0b3
Encrypted file: /content/critical/lab1/sample_file.txt
Encrypted file: /content/critical/lab1/sample_file.txt.enc, SHA-256: 119a35f1ce11ef08294a8556b5c1740edbfe487fb4a95cef4e864f0291a9e5b3
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc.enc, SHA-256: 816af57ceec1b8f0f57cf5a0f776c32a3cf66c7099b0ffa0a5f30527d0eb671e
Encrypted file: /content/critical/lab3/sample_file.txt.enc.enc
Encrypted file: /content/critical/lab3/sample_file.txt.enc.enc.enc, SHA-256: 518b2b38322a5da7548d1f5d2f6af71f9c024a584b2dafecf55795c480afa472
Encrypted file: /content/critical/lab3/sample_file.txt
Encrypted file: /content/critical/lab3/sample_file.txt.enc, SHA-256: d119c459a40feacab2db4dbf2dbae71bb463a7726b1af7bf0cc17e3c16200510

In [14]:
# To restore the files back to its original state, the AES decryption was implemented.
def decrypt_file(file_path, key):
    with open(file_path, 'rb') as file:
        # Read the initialization vector (IV) and encrypted data
        iv = file.read(16)  # IV is the first 16 bytes
        encrypted_data = file.read()

    # Initialize AES cipher with the same key and IV
    cipher = AES.new(key, AES.MODE_CBC, iv=iv)

    # Decrypt the data
    decrypted_data = cipher.decrypt(encrypted_data)

    # Remove padding
    pad_length = decrypted_data[-1]
    decrypted_data = decrypted_data[:-pad_length]

    # Save the decrypted data to a new file (correct the file extension)
    decrypted_file_path = file_path[:-4]  # Only remove the '.enc' extension
    with open(decrypted_file_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

    print(f"Decrypted {file_path} to {decrypted_file_path}")

# Decrypt the 'critical' directory
decrypt_directory('/content/critical', aes_key)


Decrypted /content/critical/lab1/sample_file.txt.enc to /content/critical/lab1/sample_file.txt
Decrypted /content/critical/lab1/sample_file.txt.enc.enc.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc.enc
Decrypted /content/critical/lab3/sample_file.txt.enc to /content/critical/lab3/sample_file.txt
Decrypted /content/critical/lab3/sample_file.txt.enc.enc.enc.enc to /content/critical/lab3/sample_file.txt.enc.enc.enc
Decrypted /content/critical/lab2/sample_file.txt.enc to /content/critical/lab2/sample_file.txt
Decrypted /content/critical/lab2/sample_file.txt.enc.enc.enc.enc to /content/critical/lab2/sample_file.txt.enc.enc.enc


In [16]:
from Crypto.Random import get_random_bytes

# Generate a valid AES key (256-bit, 32 bytes)
aes_key = get_random_bytes(32)

# This key can now be used for both encryption and decryption


In [17]:
from Crypto.Cipher import AES
import os
from Crypto.Random import get_random_bytes

# Function to decrypt a file
def decrypt_file(file_path, key):
    with open(file_path, 'rb') as file:
        # Read the initialization vector (IV) and encrypted data
        iv = file.read(16)  # IV is the first 16 bytes
        encrypted_data = file.read()

    # Initialize AES cipher with the same key and IV
    cipher = AES.new(key, AES.MODE_CBC, iv=iv)

    # Decrypt the data
    decrypted_data = cipher.decrypt(encrypted_data)

    # Remove padding
    pad_length = decrypted_data[-1]
    decrypted_data = decrypted_data[:-pad_length]

    # Save the decrypted data to a new file (remove the .enc extension)
    decrypted_file_path = file_path[:-4]  # Only remove the '.enc' extension
    with open(decrypted_file_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

    print(f"Decrypted {file_path} to {decrypted_file_path}")

# Function to decrypt all encrypted files in a directory recursively
def decrypt_directory(directory_path, key):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file_path.endswith(".enc"):  # Only decrypt encrypted files
                decrypt_file(file_path, key)

# Generate a valid AES key (256-bit, 32 bytes)
aes_key = get_random_bytes(32)

# Decrypt the 'critical' directory
decrypt_directory('/content/critical', aes_key)


Decrypted /content/critical/lab1/sample_file.txt.enc.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc
Decrypted /content/critical/lab1/sample_file.txt.enc to /content/critical/lab1/sample_file.txt
Decrypted /content/critical/lab1/sample_file.txt.enc.enc.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc.enc
Decrypted /content/critical/lab3/sample_file.txt.enc.enc.enc to /content/critical/lab3/sample_file.txt.enc.enc
Decrypted /content/critical/lab3/sample_file.txt.enc to /content/critical/lab3/sample_file.txt
Decrypted /content/critical/lab3/sample_file.txt.enc.enc.enc.enc to /content/critical/lab3/sample_file.txt.enc.enc.enc
Decrypted /content/critical/lab2/sample_file.txt.enc.enc.enc to /content/critical/lab2/sample_file.txt.enc.enc
Decrypted /content/critical/lab2/sample_file.txt.enc to /content/critical/lab2/sample_file.txt
Decrypted /content/critical/lab2/sample_file.txt.enc.enc.enc.enc to /content/critical/lab2/sample_file.txt.enc.enc.enc


In [23]:
# Function to encrypt a file
def encrypt_file(file_path, key):
    cipher = AES.new(key, AES.MODE_CBC)

    with open(file_path, 'rb') as file:
        file_data = file.read()

    # Pad the data to be a multiple of 16 bytes (AES block size)
    pad_length = 16 - len(file_data) % 16
    file_data += bytes([pad_length]) * pad_length  # Add padding

    # Encrypt the data
    encrypted_data = cipher.encrypt(file_data)

    # Ensure we add '.enc' extension only once
    if not file_path.endswith('.enc'):
        encrypted_file_path = file_path + ".enc"
    else:
        encrypted_file_path = file_path

    # Save the encrypted data to a new file with .enc extension
    with open(encrypted_file_path, 'wb') as encrypted_file:
        encrypted_file.write(cipher.iv)  # Write the IV as the first 16 bytes
        encrypted_file.write(encrypted_data)

    print(f"Encrypted {file_path} to {encrypted_file_path}")


In [25]:
# Function to encrypt a file
def encrypt_file(file_path, key):
    # Check if the file already has the .enc extension, if not, add it
    if not file_path.endswith('.enc'):
        encrypted_file_path = file_path + ".enc"
    else:
        encrypted_file_path = file_path  # If the file already has .enc, don't add it again

    # Proceed with the encryption
    cipher = AES.new(key, AES.MODE_CBC)

    with open(file_path, 'rb') as file:
        file_data = file.read()

    # Pad the data to be a multiple of 16 bytes (AES block size)
    pad_length = 16 - len(file_data) % 16
    file_data += bytes([pad_length]) * pad_length  # Add padding

    # Encrypt the data
    encrypted_data = cipher.encrypt(file_data)

    # Save the encrypted data to a new file with .enc extension
    with open(encrypted_file_path, 'wb') as encrypted_file:
        encrypted_file.write(cipher.iv)  # Write the IV as the first 16 bytes
        encrypted_file.write(encrypted_data)

    print(f"Encrypted {file_path} to {encrypted_file_path}")


In [26]:
# Function to encrypt all files in a directory recursively
def encrypt_directory(directory_path, key):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            encrypt_file(file_path, key)


In [27]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import os

# Function to encrypt a file
def encrypt_file(file_path, key):
    # Check if the file already has the .enc extension, if not, add it
    if not file_path.endswith('.enc'):
        encrypted_file_path = file_path + ".enc"
    else:
        encrypted_file_path = file_path  # If the file already has .enc, don't add it again

    # Proceed with the encryption
    cipher = AES.new(key, AES.MODE_CBC)

    with open(file_path, 'rb') as file:
        file_data = file.read()

    # Pad the data to be a multiple of 16 bytes (AES block size)
    pad_length = 16 - len(file_data) % 16
    file_data += bytes([pad_length]) * pad_length  # Add padding

    # Encrypt the data
    encrypted_data = cipher.encrypt(file_data)

    # Save the encrypted data to a new file with .enc extension
    with open(encrypted_file_path, 'wb') as encrypted_file:
        encrypted_file.write(cipher.iv)  # Write the IV as the first 16 bytes
        encrypted_file.write(encrypted_data)

    print(f"Encrypted {file_path} to {encrypted_file_path}")

# Function to encrypt all files in a directory recursively
def encrypt_directory(directory_path, key):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            encrypt_file(file_path, key)

# Generate a valid AES key (256-bit, 32 bytes)
aes_key = get_random_bytes(32)

# Encrypt the directory
encrypt_directory('/content/critical', aes_key)


Encrypted /content/critical/lab1/sample_file.txt.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc
Encrypted /content/critical/lab1/sample_file.txt.enc.enc.enc.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc.enc.enc.enc
Encrypted /content/critical/lab1/sample_file.txt to /content/critical/lab1/sample_file.txt.enc
Encrypted /content/critical/lab1/sample_file.txt.enc.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc.enc
Encrypted /content/critical/lab1/sample_file.txt.enc to /content/critical/lab1/sample_file.txt.enc
Encrypted /content/critical/lab1/sample_file.txt.enc.enc.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc.enc.enc
Encrypted /content/critical/lab3/sample_file.txt.enc.enc to /content/critical/lab3/sample_file.txt.enc.enc
Encrypted /content/critical/lab3/sample_file.txt.enc.enc.enc.enc.enc to /content/critical/lab3/sample_file.txt.enc.enc.enc.enc.enc
Encrypted /content/critical/lab3/sample_file.txt to /content/critical/lab3/sample_file.txt.e

In [32]:
# Function to encrypt a file
def encrypt_file(file_path, key):
    # Generate a random IV for encryption
    cipher = AES.new(key, AES.MODE_CBC)

    with open(file_path, 'rb') as file:
        file_data = file.read()

    # Pad the data to be a multiple of 16 bytes (AES block size)
    pad_length = 16 - len(file_data) % 16
    file_data += bytes([pad_length]) * pad_length  # Add padding

    # Encrypt the data
    encrypted_data = cipher.encrypt(file_data)

    # Ensure we add '.enc' extension only once
    if not file_path.endswith('.enc'):
        encrypted_file_path = file_path + ".enc"
    else:
        encrypted_file_path = file_path  # If the file already has .enc, don't add it again

    # Save the encrypted data to a new file with .enc extension
    with open(encrypted_file_path, 'wb') as encrypted_file:
        encrypted_file.write(cipher.iv)  # Write the IV as the first 16 bytes
        encrypted_file.write(encrypted_data)

    print(f"Encrypted {file_path} to {encrypted_file_path}")


In [33]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import os

# Function to encrypt a file
def encrypt_file(file_path, key):
    # Generate a random IV for encryption
    cipher = AES.new(key, AES.MODE_CBC)

    with open(file_path, 'rb') as file:
        file_data = file.read()

    # Pad the data to be a multiple of 16 bytes (AES block size)
    pad_length = 16 - len(file_data) % 16
    file_data += bytes([pad_length]) * pad_length  # Add padding

    # Encrypt the data
    encrypted_data = cipher.encrypt(file_data)

    # Ensure we add '.enc' extension only once
    if not file_path.endswith('.enc'):
        encrypted_file_path = file_path + ".enc"
    else:
        encrypted_file_path = file_path  # If the file already has .enc, don't add it again

    # Save the encrypted data to a new file with .enc extension
    with open(encrypted_file_path, 'wb') as encrypted_file:
        encrypted_file.write(cipher.iv)  # Write the IV as the first 16 bytes
        encrypted_file.write(encrypted_data)

    print(f"Encrypted {file_path} to {encrypted_file_path}")

# Function to encrypt all files in a directory recursively
def encrypt_directory(directory_path, key):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            encrypt_file(file_path, key)

# Generate a valid AES key (256-bit, 32 bytes)
aes_key = get_random_bytes(32)

# Encrypt the directory
encrypt_directory('/content/critical', aes_key)


Encrypted /content/critical/lab1/sample_file.txt.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc
Encrypted /content/critical/lab1/sample_file.txt.enc.enc.enc.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc.enc.enc.enc
Encrypted /content/critical/lab1/sample_file.txt to /content/critical/lab1/sample_file.txt.enc
Encrypted /content/critical/lab1/sample_file.txt.enc.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc.enc
Encrypted /content/critical/lab1/sample_file.txt.enc to /content/critical/lab1/sample_file.txt.enc
Encrypted /content/critical/lab1/sample_file.txt.enc.enc.enc.enc to /content/critical/lab1/sample_file.txt.enc.enc.enc.enc
Encrypted /content/critical/lab3/sample_file.txt.enc.enc to /content/critical/lab3/sample_file.txt.enc.enc
Encrypted /content/critical/lab3/sample_file.txt.enc.enc.enc.enc.enc to /content/critical/lab3/sample_file.txt.enc.enc.enc.enc.enc
Encrypted /content/critical/lab3/sample_file.txt to /content/critical/lab3/sample_file.txt.e

In [50]:
# prompt: solve all the errors above

import os
import hashlib
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# Function to decrypt a file
def decrypt_file(file_path, key):
    with open(file_path, 'rb') as file:
        iv = file.read(16)  # The first 16 bytes are the IV
        print(f"IV for {file_path}: {iv}")  # Debugging: Print the IV

        if len(iv) != 16:
            raise ValueError(f"Invalid IV length in {file_path}, it must be 16 bytes.")

        encrypted_data = file.read()  # The rest is the encrypted data

    # Initialize AES cipher with the same key and IV
    cipher = AES.new(key, AES.MODE_CBC, iv=iv)

    # Decrypt the data
    decrypted_data = cipher.decrypt(encrypted_data)

    # Remove padding (the last byte indicates the padding length)
    pad_length = decrypted_data[-1]
    decrypted_data = decrypted_data[:-pad_length]  # Remove padding

    # Save the decrypted data to a new file (remove the .enc extension)
    decrypted_file_path = file_path[:-4]  # Only remove the '.enc' extension
    with open(decrypted_file_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

    print(f"Decrypted {file_path} to {decrypted_file_path}")

# Function to decrypt all encrypted files in a directory recursively
def decrypt_directory(directory_path, key):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file_path.endswith(".enc"):  # Only decrypt encrypted files
                decrypt_file(file_path, key)

# Use the same AES key as before for decryption (must be the same key used for encryption)
# aes_key = get_random_bytes(32)  # Replace this with the key used for encryption

print("Current working directory:", os.getcwd())

# Created the "critical" directory with subdirectories and sample files

base_dir = "/content/critical"
sub_dirs = ["lab1", "lab2", "lab3"]

if not os.path.exists(base_dir):
    os.makedirs(base_dir)

for sub_dir in sub_dirs:
    sub_dir_path = os.path.join(base_dir, sub_dir)
    os.makedirs(sub_dir_path, exist_ok=True)
    with open(os.path.join(sub_dir_path, "sample_file.txt"), "w") as f:
        f.write("This is a sample file for encryption.")

print("Directory structure created successfully in /content.")

!pip install pycryptodome


# Implemented AES encryption to encrypt all files inside the critical directory.

def encrypt_file(file_path, key):
    """Encrypts a file using AES in CBC mode."""
    try:
        cipher = AES.new(key, AES.MODE_CBC)
        with open(file_path, "rb") as f:
            data = f.read()

        pad_length = 16 - (len(data) % 16)
        data += bytes([pad_length]) * pad_length

        encrypted_data = cipher.encrypt(data)

        with open(file_path + ".enc", "wb") as f:
            f.write(cipher.iv)  # Write the IV first
            f.write(encrypted_data)

        os.remove(file_path)
        print(f"Encrypted file: {file_path}")
    except Exception as e:
        print(f"Error encrypting {file_path}: {e}")


def decrypt_file(file_path, key):
    """Decrypts a file using AES in CBC mode."""
    try:
        with open(file_path, "rb") as f:
            iv = f.read(16)
            encrypted_data = f.read()

        cipher = AES.new(key, AES.MODE_CBC, iv=iv)
        decrypted_data = cipher.decrypt(encrypted_data)

        padding_length = decrypted_data[-1]
        decrypted_data = decrypted_data[:-padding_length]

        with open(file_path[:-4], "wb") as f:
            f.write(decrypted_data)

        os.remove(file_path)
        print(f"Decrypted file: {file_path}")
    except Exception as e:
        print(f"Error decrypting {file_path}: {e}")


def hash_file(file_path):
    """Calculates the SHA-256 hash of a file."""
    hasher = hashlib.sha256()
    with open(file_path, "rb") as f:
        while True:
            chunk = f.read(4096)
            if not chunk:
                break
            hasher.update(chunk)
    return hasher.hexdigest()

# Example usage:
aes_key = get_random_bytes(32)
base_dir = "/content/critical"

for root, dirs, files in os.walk(base_dir):
    for filename in files:
        file_path = os.path.join(root, filename)
        if os.path.isfile(file_path):
            encrypt_file(file_path, aes_key)
            encrypted_file_path = file_path + ".enc"
            if os.path.exists(encrypted_file_path):
                encrypted_file_hash = hash_file(encrypted_file_path)
                print(f"Encrypted file: {encrypted_file_path}, SHA-256: {encrypted_file_hash}")



# Generate a valid AES key (256-bit, 32 bytes)
aes_key = get_random_bytes(32)

# Decrypt the 'critical' directory
decrypt_directory('/content/critical', aes_key)


Current working directory: /content
Directory structure created successfully in /content.
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc.enc.enc
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc.enc.enc.enc, SHA-256: 415cb8889f86e53c7092f6e4583bbd808a0444825c561c1f40ffc0273cd396b2
Encrypted file: /content/critical/lab1/sample_file.txt
Encrypted file: /content/critical/lab1/sample_file.txt.enc, SHA-256: a2519209dc9e497ae316c3fcef4d98577bf7429998c7525ba484917bc19eb785
Encrypted file: /content/critical/lab1/sample_file.txt.enc
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc, SHA-256: 34c40f71b7d2c94b929afa6077cc5acde184cccb81057c5bf84298a3aa40c511
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc.enc
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc.enc.enc, SHA-256: 49a855a9616ef4b69d2899cf844df33e45e55029d2bb266a80cfac24fe302180
Encrypted file: /content/critical/lab3/sample_file.txt.enc.enc.enc.enc.enc

In [41]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import os

# Function to encrypt a file
def encrypt_file(file_path, key):
    # Generate a random IV for encryption
    cipher = AES.new(key, AES.MODE_CBC)

    with open(file_path, 'rb') as file:
        file_data = file.read()

    # Pad the data to be a multiple of 16 bytes (AES block size)
    pad_length = 16 - len(file_data) % 16
    file_data += bytes([pad_length]) * pad_length  # Add padding

    # Encrypt the data
    encrypted_data = cipher.encrypt(file_data)

    # Ensure we add '.enc' extension only once
    if not file_path.endswith('.enc'):
        encrypted_file_path = file_path + ".enc"
    else:
        encrypted_file_path = file_path  # If the file already has .enc, don't add it again

    # Save the encrypted data to a new file with .enc extension
    with open(encrypted_file_path, 'wb') as encrypted_file:
        encrypted_file.write(cipher.iv)  # Write the IV as the first 16 bytes
        encrypted_file.write(encrypted_data)

    print(f"Encrypted {file_path} to {encrypted_file_path}")

# Function to encrypt all files in a directory recursively
def encrypt_directory(directory_path, key):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            # Only encrypt files that don't already have .enc extension
            if not file_path.endswith('.enc'):
                encrypt_file(file_path, key)

# Generate a valid AES key (256-bit, 32 bytes)
aes_key = get_random_bytes(32)

# Encrypt the directory
encrypt_directory('/content/critical', aes_key)


In [49]:
# decryption and encrption

import os
import hashlib
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# Function to encrypt a file
def encrypt_file(file_path, key):
    """Encrypts a file using AES in CBC mode."""
    try:
        iv = get_random_bytes(16)  # Generate a new IV for each file
        cipher = AES.new(key, AES.MODE_CBC, iv=iv)
        with open(file_path, "rb") as f:
            data = f.read()

        pad_length = 16 - (len(data) % 16)
        data += bytes([pad_length]) * pad_length

        encrypted_data = cipher.encrypt(data)

        with open(file_path + ".enc", "wb") as f:
            f.write(iv)  # Write the IV first
            f.write(encrypted_data)

        os.remove(file_path)
        print(f"Encrypted file: {file_path}")
    except Exception as e:
        print(f"Error encrypting {file_path}: {e}")


def decrypt_file(file_path, key):
    """Decrypts a file using AES in CBC mode."""
    try:
        with open(file_path, "rb") as f:
            iv = f.read(16)
            encrypted_data = f.read()

        cipher = AES.new(key, AES.MODE_CBC, iv=iv)
        decrypted_data = cipher.decrypt(encrypted_data)

        padding_length = decrypted_data[-1]
        decrypted_data = decrypted_data[:-padding_length]

        with open(file_path[:-4], "wb") as f:
            f.write(decrypted_data)

        os.remove(file_path)
        print(f"Decrypted file: {file_path}")
    except Exception as e:
        print(f"Error decrypting {file_path}: {e}")


def hash_file(file_path):
    """Calculates the SHA-256 hash of a file."""
    hasher = hashlib.sha256()
    with open(file_path, "rb") as f:
        while True:
            chunk = f.read(4096)
            if not chunk:
                break
            hasher.update(chunk)
    return hasher.hexdigest()

# Example usage:
aes_key = get_random_bytes(32)
base_dir = "/content/critical"

if not os.path.exists(base_dir):
    os.makedirs(base_dir)

for root, dirs, files in os.walk(base_dir):
    for filename in files:
        file_path = os.path.join(root, filename)
        if os.path.isfile(file_path):
            encrypt_file(file_path, aes_key)
            encrypted_file_path = file_path + ".enc"
            if os.path.exists(encrypted_file_path):
                encrypted_file_hash = hash_file(encrypted_file_path)
                print(f"Encrypted file: {encrypted_file_path}, SHA-256: {encrypted_file_hash}")


# Generate a valid AES key (256-bit, 32 bytes)
# aes_key = get_random_bytes(32)  
# The same decryption key can be used for decryption
# aes_key from the encryption part should be unchanged

# Decrypt the 'critical' directory
decrypt_directory('/content/critical', aes_key)


Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc.enc.enc
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc.enc.enc.enc, SHA-256: ce54655b3cdb2f808135f6de8c8a9b3204b8d8af4ad0b20dfd110e03d794a414
Encrypted file: /content/critical/lab1/sample_file.txt.enc
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc, SHA-256: 8a1b32b3d7314534adb15c6453594cacf3a428bd2dd258d37e592ba688baefdb
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc.enc
Encrypted file: /content/critical/lab1/sample_file.txt.enc.enc.enc.enc.enc, SHA-256: e1b2528f871c198ab31349f117cbdc5d40938e3ca96ee5088a982e97a4f7367b
Encrypted file: /content/critical/lab3/sample_file.txt.enc.enc.enc.enc.enc
Encrypted file: /content/critical/lab3/sample_file.txt.enc.enc.enc.enc.enc.enc, SHA-256: ba2763c75451c8f92ff6f5aea00f358e0c8b7b38e2a98e82b3b890c26a0a7e9b
Encrypted file: /content/critical/lab3/sample_file.txt.enc
Encrypted file: /content/critical/lab3/sample_file.txt.enc.enc, S